In [0]:
sales_df= df = spark.read.csv('/FileStore/tables/sales.csv', header=True, inferSchema=True)

sales_df.show()

+----------+-----------+----------+--------+------------+
|product_id|customer_id|order_date|location|source_order|
+----------+-----------+----------+--------+------------+
|         1|          A|2023-01-01|   India|      Swiggy|
|         2|          A|2022-01-01|   India|      Swiggy|
|         2|          A|2023-01-07|   India|      Swiggy|
|         3|          A|2023-01-10|   India|  Restaurant|
|         3|          A|2022-01-11|   India|      Swiggy|
|         3|          A|2023-01-11|   India|  Restaurant|
|         2|          B|2022-02-01|   India|      Swiggy|
|         2|          B|2023-01-02|   India|      Swiggy|
|         1|          B|2023-01-04|   India|  Restaurant|
|         1|          B|2023-02-11|   India|      Swiggy|
|         3|          B|2023-01-16|   India|      zomato|
|         3|          B|2022-02-01|   India|      zomato|
|         3|          C|2023-01-01|   India|      zomato|
|         1|          C|2023-01-01|      UK|      Swiggy|
|         6|  

In [0]:
sales_df.printSchema()

root
 |-- product_id: integer (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_date: date (nullable = true)
 |-- location: string (nullable = true)
 |-- source_order: string (nullable = true)



In [0]:
from pyspark.sql.functions import month,year,quarter

sales_df = sales_df.withColumn("order_year",year(sales_df.order_date))

sales_df = sales_df.withColumn("order_month",month(sales_df.order_date))

sales_df = sales_df.withColumn("order_quater",quarter(sales_df.order_date))

sales_df.show()

+----------+-----------+----------+--------+------------+----------+-----------+------------+
|product_id|customer_id|order_date|location|source_order|order_year|order_month|order_quater|
+----------+-----------+----------+--------+------------+----------+-----------+------------+
|         1|          A|2023-01-01|   India|      Swiggy|      2023|          1|           1|
|         2|          A|2022-01-01|   India|      Swiggy|      2022|          1|           1|
|         2|          A|2023-01-07|   India|      Swiggy|      2023|          1|           1|
|         3|          A|2023-01-10|   India|  Restaurant|      2023|          1|           1|
|         3|          A|2022-01-11|   India|      Swiggy|      2022|          1|           1|
|         3|          A|2023-01-11|   India|  Restaurant|      2023|          1|           1|
|         2|          B|2022-02-01|   India|      Swiggy|      2022|          2|           1|
|         2|          B|2023-01-02|   India|      Swiggy|   

In [0]:
menu_df= df = spark.read.csv('/FileStore/tables/menu.csv', header=True, inferSchema=True)

menu_df.show()

+----------+------------+-----+
|product_id|product_name|price|
+----------+------------+-----+
|         1|      PIZZA,|  100|
|         2|    Chowmin,|  150|
|         3|   sandwich,|  120|
|         4|       Dosa,|  110|
|         5|    Biryani,|   80|
|         6|      Pasta,|  180|
+----------+------------+-----+



#KPI s

## Total amount spend by each customer

In [0]:
total_spend = (sales_df.join(menu_df,'product_id').groupBy('customer_id').agg({'price':'sum'}).orderBy('customer_id'))

display(total_spend)

customer_id,sum(price)
A,4260
B,4440
C,2400
D,1200
E,2040


Databricks visualization. Run in Databricks to view.

## total amount spend by each food category

In [0]:
total_spend = (sales_df.join(menu_df,'product_id').groupBy('product_name').agg({'price':'sum'}).orderBy('product_name'))

display(total_spend)

product_name,sum(price)
"Biryani,",480
"Chowmin,",3600
"Dosa,",1320
"PIZZA,",2100
"Pasta,",1080
"sandwich,",5760


Databricks visualization. Run in Databricks to view.

## total amount of sales in each month

In [0]:
month_sales = (sales_df.join(menu_df,'product_id').groupBy('order_month').agg({'price':'sum'}).orderBy('order_month'))

display(month_sales)

order_month,sum(price)
1,2960
2,2730
3,910
5,2960
6,2960
7,910
11,910


Databricks visualization. Run in Databricks to view.

## yearly sales

In [0]:
year_sales = (sales_df.join(menu_df,'product_id').groupBy('order_year').agg({'price':'sum'}).orderBy('order_year'))

display(year_sales)

order_year,sum(price)
2022,4350
2023,9990


Databricks visualization. Run in Databricks to view.

## Quartly sales

In [0]:
quator_sales = (sales_df.join(menu_df,'product_id').groupBy('order_quater').agg({'price':'sum'}).orderBy('order_quater'))

display(quator_sales)

order_quater,sum(price)
1,6600
2,5920
3,910
4,910


Databricks visualization. Run in Databricks to view.

## how many times each product purchased 

In [0]:
from pyspark.sql.functions import count

most_df = (sales_df.join(menu_df,'product_id').groupBy('product_id','product_name').agg(count('product_id').alias('product_count')).orderBy('product_count',ascending = 0).drop('product_id'))

display(most_df)

product_name,product_count
"sandwich,",48
"Chowmin,",24
"PIZZA,",21
"Dosa,",12
"Biryani,",6
"Pasta,",6


Databricks visualization. Run in Databricks to view.

## top 5 order items

In [0]:
most_df = (sales_df.join(menu_df,'product_id').groupBy('product_id','product_name').agg(count('product_id').alias('product_count')).orderBy('product_count',ascending = 0).drop('product_id').limit(5)
           )

display(most_df)

product_name,product_count
"sandwich,",48
"Chowmin,",24
"PIZZA,",21
"Dosa,",12
"Biryani,",6


## most order item

In [0]:
most_df = (sales_df.join(menu_df,'product_id').groupBy('product_id','product_name').agg(count('product_id').alias('product_count')).orderBy('product_count',ascending = 0).drop('product_id').limit(1)
           )

display(most_df)

product_name,product_count
"sandwich,",48


Databricks visualization. Run in Databricks to view.

## sales by country


In [0]:
by_country = (sales_df.join(menu_df,'product_id').groupBy('location').agg({'price':'sum'})
              )

display(by_country)

location,sum(price)
India,4860
USA,2460
UK,7020


Databricks visualization. Run in Databricks to view.

## total sales by order source

In [0]:
by_source = (sales_df.join(menu_df,'product_id').groupBy('source_order').agg({'price':'sum'})
              )

display(by_source)

source_order,sum(price)
zomato,4920
Swiggy,6330
Restaurant,3090


Databricks visualization. Run in Databricks to view.